In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.autograd import Variable
import torch.nn.utils.prune as prune

import os
import copy

In [ ]:
# Device configuration-
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"currently available device: {device}")

currently available device: cuda


In [ ]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError(
                'BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError(
                "Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

def resnet_18(num_classes):

    model = ResNet(block=BasicBlock, layers=[2, 2, 2, 2], num_classes=10)

    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1,
                            bias=False)

    return model

In [ ]:
# load the data
train_transform = transforms.Compose(
    [
     transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))])

test_transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))])

ds = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)


test_ds = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)

# split the training set and validation set
torch.manual_seed(50)
test_size = len(test_ds)
val_size = 5000
train_size = len(ds) - val_size
batch_size = 512

train_ds, val_ds = random_split(ds, [train_size, val_size])


train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)

val_loader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False)

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

classes = ds.classes

In [ ]:
import time

def validate(model, criterion, val_loader, use_gpu=False):
  val_size = len(val_loader.dataset)
  val_loss = 0
  correct = 0
  device = torch.device( "cuda:0" if use_gpu else "cpu" )

  with torch.no_grad():
    for i, data in enumerate(val_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # forward + backward + optimize
        outputs = model(inputs).to(device)
        loss = criterion(outputs, labels)
        
        val_loss += loss * inputs.size(0)
        
        # val accuracy
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

    val_loss = val_loss/val_size
    val_accuracy = correct/val_size;

  return val_loss, val_accuracy

def get_train_accuracy(model, criterion, train_loader, use_gpu=False):
   train_size = len(train_loader.dataset)
   correct = 0
   device = torch.device( "cuda:0" if use_gpu else "cpu" )
   
   with torch.no_grad():
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # forward + backward + optimize
        outputs = model(inputs).to(device)

        # val accuracy
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

    print(correct)
    return correct/train_size

def train(model, criterion, optimizer, train_loader, val_loader, epoch=50, use_gpu=True):

  train_size = len(train_loader.dataset)
  device = torch.device( "cuda:0" if use_gpu else "cpu" )
  
  history = {
      'train_loss': [], 
      'train_accuracy': [], 
      'val_loss': [], 
      'val_accuracy': []
      } 

  model = model.to(device)

  for epoch in range(epoch):  # loop over the dataset multiple times
    running_loss = 0.0
    correct = 0
    
    print(f'------------------------------\n Epoch: {epoch + 1}')
    
    t1 = time.time()
    for  i,data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # as loss.item() return the average batch loss, so convert it to the total loss
        running_loss += loss.item() * inputs.size(0)
           
    t2 = time.time()
    t = t2 - t1

    epoch_train_loss = running_loss/train_size
    epoch_train_accuracy = get_train_accuracy(model, criterion, train_loader, use_gpu)
    epoch_val_loss, epoch_val_accuracy = validate(model, criterion, val_loader, use_gpu)
    print(f'time: {int(t)}sec train_loss: {epoch_train_loss}, train_accuracy: {epoch_train_accuracy}, val_loss: {epoch_val_loss}, val_accuracy: {epoch_val_accuracy}');
  
    history['train_loss'].append(epoch_train_loss)
    history['train_accuracy'].append(epoch_train_accuracy)
    history['val_loss'].append(epoch_val_loss)
    history['val_accuracy'].append(epoch_val_accuracy)
 
  return history

In [ ]:
# run test set
def test(model, criterion, test_loader, use_gpu=False):
  test_size = len(test_loader.dataset)
  device = torch.device( "cuda:0" if use_gpu else "cpu" )
  test_loss = 0.0
  test_accuracy = 0
  correct = 0
  model.eval()
  with torch.no_grad():
      for i, data in enumerate(test_loader):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data[0].to(device), data[1].to(device)

          # forward + backward + optimize
          outputs = model(inputs).to(device)
          loss = criterion(outputs, labels)
          
          test_loss += loss * inputs.size(0)
          
          # val accuracy
          _, predicted = torch.max(outputs.data, 1)
          correct += (predicted == labels).sum().item()

      
      test_loss = test_loss/test_size
      test_accuracy = correct/test_size;

  return test_loss, test_accuracy

In [ ]:
# Helper function to calculate sparsity of a module

def measure_module_sparsity(module, weight=True, bias=False, use_mask=False):

    num_zeros = 0
    num_elements = 0

    if use_mask == True:
        for buffer_name, buffer in module.named_buffers():
            if "weight_mask" in buffer_name and weight == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
            if "bias_mask" in buffer_name and bias == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
    else:
        for param_name, param in module.named_parameters():
            if "weight" in param_name and weight == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()
            if "bias" in param_name and bias == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()

    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity

In [ ]:
import torch.nn.utils.prune as prune

# FUnction for performing unstructured pruning

def pruning_func_unstruct(model, a, g=False):
    if g == True:
        # Global pruning
        parameters_to_prune = []
        for module_name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                parameters_to_prune.append((module, "weight"))

        prune.global_unstructured(
            parameters_to_prune,
            pruning_method=prune.L1Unstructured,
            amount=a,
        )
        for module_name, module in model.named_modules():
          if isinstance(module, torch.nn.Conv2d):
              prune.remove(module, 'weight')
              pass
    else:
        for module_name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                prune.l1_unstructured(module,
                                      name="weight",
                                      amount=a)
                prune.remove(module, 'weight')
            elif isinstance(module, torch.nn.Linear):
                pass

    return model

In [ ]:
# Function for performing unstructured pruning
def pruning_func_struct(model, a, norm=2):

  for module_name, module in model.named_modules():
    if 'layer1' in module_name or 'layer2' in module_name or 'layer3' in module_name or 'layer4' in module_name:
      if isinstance(module, torch.nn.Conv2d):
        prune.ln_structured(module, name="weight", amount=a, n=norm, dim=0)
        prune.remove(module, 'weight')

  return model

In [ ]:
# Function to calculate sparsity in the whole model
def measure_global_sparsity(
    model, weight = True,
    bias = False, conv2d_use_mask = False,
    linear_use_mask = False):

    num_zeros = 0
    num_elements = 0

    for module_name, module in model.named_modules():

        if isinstance(module, torch.nn.Conv2d):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=conv2d_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

        elif isinstance(module, torch.nn.Linear):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=linear_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity

In [ ]:
# Function to stop re-parametrization of pruned neurons

def remove_parameters(model):

    for module_name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            try:
                prune.remove(module, "weight")
            except:
                pass
            try:
                prune.remove(module, "bias")
            except:
                pass
        elif isinstance(module, torch.nn.Linear):
            try:
                prune.remove(module, "weight")
            except:
                pass
            try:
                prune.remove(module, "bias")
            except:
                pass

    return model

In [ ]:
trained_model = resnet_18(10).to(device)

optimizer = optim.SGD(trained_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

checkpoint = torch.load('/content/drive/Shareddrives/RDNN/resnet18_baseline_cifar10.pth')

trained_model.load_state_dict(checkpoint['state_dict'])

In [ ]:
for module_name, module in trained_model.named_modules():
  if 'layer1' in module_name or 'layer2' in module_name or 'layer3' in module_name or 'layer4' in module_name:
    print(module_name)

In [ ]:
# Prune and finetune the models

def pruning_finetuning(model,
                                 amount=0.4,
                                 num_iterations=1,
                                 num_epochs_per_iteration=2,
                                 structured = False,
                                 global_pruning=False):

    for i in range(num_iterations):

        print("Pruning and Finetuning {}/{}".format(i + 1, num_iterations))
        print("Pruning...")

        if structured == False:
          model = pruning_func_unstruct(model, amount, global_pruning)

        else:
          model = pruning_func_struct(model, amount, norm=2)

        print("Fine-tuning...")

        history = train(model, criterion, optimizer, train_loader, val_loader, num_epochs_per_iteration, torch.cuda.is_available())

    return model,history

In [ ]:
pruned = copy.deepcopy(trained_model)

In [ ]:
measure_global_sparsity(
    pruned, weight = True,
    bias = False, conv2d_use_mask = False,
    linear_use_mask = False)

In [ ]:
pruned , history = pruning_finetuning(pruned,
                  amount=0.9,
                  num_iterations=1,
                  num_epochs_per_iteration=1,
                  structured = True,
                  global_pruning=False)

pruned = remove_parameters(model=pruned)
print(history)

In [ ]:
history

In [ ]:
measure_global_sparsity(
    pruned, weight = True,
    bias = False, conv2d_use_mask = False,
    linear_use_mask = False)

In [ ]:
torch.save({'loss_history' : history, 'state_dict': pruned.state_dict()}, '/content/p.pth')

In [ ]:
model = resnet_18(10).to(device)
checkpoint1 = torch.load('/content/p.pth')
model.load_state_dict(checkpoint1['state_dict'])

In [ ]:
measure_global_sparsity(
    model, weight = True,
    bias = False, conv2d_use_mask = False,
    linear_use_mask = False)

In [ ]:
# Global Unstructured Pruning

spar = [0.1,0.4,0.6,0.75,0.9]

trained_model = resnet_18(10).to(device)
optimizer = optim.SGD(trained_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
checkpoint = torch.load('/content/drive/Shareddrives/RDNN/resnet18_baseline_cifar10.pth')
trained_model.load_state_dict(checkpoint['state_dict'])

for i in range(len(spar)):
  pruned = copy.deepcopy(trained_model)

  pruned , history = pruning_finetuning(pruned,
                    amount=spar[i],
                    num_iterations=1,
                    num_epochs_per_iteration=10,
                    structured = False,
                    global_pruning=True)
  
  pruned = remove_parameters(model=pruned)

  print(measure_global_sparsity(
      pruned, weight = True,
      bias = False, conv2d_use_mask = False,
      linear_use_mask = False))

  torch.save({'loss_history' : history, 'state_dict': pruned.state_dict()},f'/content/drive/Shareddrives/RDNN/Pruned_Models/global_unstruct_sparsity_{spar[i]}.pth')

In [ ]:
# Local Unstructured Pruning

spar = [0.1,0.4,0.6,0.75,0.9]

trained_model = resnet_18(10).to(device)
optimizer = optim.SGD(trained_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
checkpoint = torch.load('/content/drive/Shareddrives/RDNN/resnet18_baseline_cifar10.pth')
trained_model.load_state_dict(checkpoint['state_dict'])

for i in range(len(spar)):
  pruned = copy.deepcopy(trained_model)

  pruned , history = pruning_finetuning(pruned,
                    amount=spar[i],
                    num_iterations=1,
                    num_epochs_per_iteration=10,
                    structured = False,
                    global_pruning=False)
  
  pruned = remove_parameters(model=pruned)

  print(measure_global_sparsity(
      pruned, weight = True,
      bias = False, conv2d_use_mask = False,
      linear_use_mask = False))

  torch.save({'loss_history' : history, 'state_dict': pruned.state_dict()},f'/content/drive/Shareddrives/RDNN/Pruned_Models/local_unstruct_sparsity_{spar[i]}.pth')

In [ ]:
# Structured Pruning

# Local Unstructured Pruning

spar = [0.9,0.75,0.6,0.4,0.1]

trained_model = resnet_18(10).to(device)
optimizer = optim.SGD(trained_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
checkpoint = torch.load('/content/drive/Shareddrives/RDNN/resnet18_baseline_cifar10.pth')
trained_model.load_state_dict(checkpoint['state_dict'])

for i in range(len(spar)):
  pruned = copy.deepcopy(trained_model)

  pruned , history = pruning_finetuning(pruned,
                    amount=spar[i],
                    num_iterations=1,
                    num_epochs_per_iteration=15,
                    structured = True,
                    global_pruning=False)
  
  pruned = remove_parameters(model=pruned)

  print(measure_global_sparsity(
      pruned, weight = True,
      bias = False, conv2d_use_mask = False,
      linear_use_mask = False))

  torch.save({'loss_history' : history, 'state_dict': pruned.state_dict()},f'/content/drive/Shareddrives/RDNN/Pruned_Models/structured_sparsity_{spar[i]}.pth')